In [13]:
import tkinter as tk
from tkinter import ttk
import sqlite3
from PIL import Image, ImageTk

# Inicializar a variável global 'conn'
conn = None

# Inicializar a variável global 'imagem_label'
imagem_label = None

# Definir a lista 'labels' globalmente
labels = ["ID Solicitaçao", "Local", "Colaborador", "Setor", "Cargo", "Data Solicitação", "Hora Solicitação",
          "Responsável", "Nro NF", "Data Compra", "VL Total", "Itens Comprados", "VL Item", "Comprovante"]

def exibir_imagem(comprovante_nome):
    global imagem_label  # Certifique-se de que 'imagem_label' seja global
    
    try:
        imagem_path = f"Comprovantes/{comprovante_nome}"  # Caminho completo para a imagem
        imagem = Image.open(imagem_path)
        imagem = imagem.resize((300, 300), Image.ANTIALIAS)
        imagem = ImageTk.PhotoImage(imagem)
        
        imagem_label.config(image=imagem)
        imagem_label.image = imagem  # Mantenha uma referência para evitar a coleta de lixo
    except Exception as e:
        # Trate qualquer exceção que possa ocorrer ao carregar a imagem
        print(f"Erro ao carregar imagem: {e}")

def conectar_ao_banco():
            conn = sqlite3.connect("Controle_de_Caixa.db")
            cursor = conn.cursor()

            # Verificar se a tabela já existe, se não, crie-a
            cursor.execute('''
                CREATE TABLE IF NOT EXISTS controle_caixa (
                    id INTEGER PRIMARY KEY,
                    ID_LAN TEXT,  -- Adicione esta linha para criar a coluna ID_LAN
                    Local TEXT,
                    Colaborador TEXT,
                    Setor TEXT,
                    Cargo TEXT,
                    Data_Solicitacao TEXT,
                    Hora_Solicitacao TEXT,
                    Responsavel TEXT,
                    Nro_NF TEXT,
                    Data_Compra TEXT,
                    VL_Total TEXT,
                    Itens_Comprados TEXT,
                    VL_Item TEXT,
                    Comprovante TEXT
                )
            ''')

            conn.commit()
            return conn

def main():  
     
    # Conectar ao banco de dados SQLite
    global conn
    global imagem_label
    conn = conectar_ao_banco()  
    
    def consultar_lancamentos():
                
        # Cabeçalho da imagem
        imagem_label = ttk.Label(root)
        imagem_label.place(x=10, y=10, width=300, height=300)
        
        def selecionar_resultado(event):
            item_selecionado = resultados_tree.selection()[0]  # Obtém o item selecionado
            comprovante_nome = resultados_tree.item(item_selecionado, "values")[-1]  # Obtém o nome do arquivo comprovante na última coluna
        
            # Chama a função para exibir a imagem
            exibir_imagem(comprovante_nome)
        
        def buscar_lancamentos():
            # Obter as opções selecionadas (somente uma deve estar selecionada)
            opcao_selecionada = opcao_combobox.get()
            
            # Obter o texto de pesquisa
            texto_pesquisa = pesquisa_entry.get()
            
            # Verificar se a conexão está inicializada
            if not conn:
                # Lógica para lidar com a conexão não inicializada
                return
            
            # Construir a consulta SQL com base na opção selecionada
            if opcao_selecionada == "Código":
                query = f"SELECT * FROM controle_caixa WHERE ID_LAN LIKE '%{texto_pesquisa}%'"
            elif opcao_selecionada == "Colaborador":
                query = f"SELECT * FROM controle_caixa WHERE Colaborador LIKE '%{texto_pesquisa}%'"
            elif opcao_selecionada == "Data":
                query = f"SELECT * FROM controle_caixa WHERE Data_Solicitacao LIKE '%{texto_pesquisa}%'"
            else:
                return
            
            # Limpar a treeview existente
            for row in resultados_tree.get_children():
                resultados_tree.delete(row)
            
            # Executar a consulta e exibir os resultados na treeview
            cursor = conn.execute(query)
            for row in cursor.fetchall():
                resultados_tree.insert("", "end", values=row[1:])

        
        # Criar uma nova janela para pesquisa
        pesquisa_window = tk.Toplevel(root)
        pesquisa_window.title("Consultar/Buscar Lançamentos")
        
        # Rótulo de instrução
        instrucao_label = ttk.Label(pesquisa_window, text="Selecione uma opção e insira o texto de pesquisa:")
        instrucao_label.pack(padx=10, pady=10)
        
        # Caixa de combinação para seleção de opção
        opcao_combobox = ttk.Combobox(pesquisa_window, values=["Código", "Colaborador", "Data"])
        opcao_combobox.pack(padx=10, pady=5)
        opcao_combobox.set("Código")
        
        # Entrada de texto para pesquisa
        pesquisa_entry = ttk.Entry(pesquisa_window, width=30)
        pesquisa_entry.pack(padx=10, pady=5)
        
        # Treeview para exibir resultados
        resultados_tree = ttk.Treeview(pesquisa_window, columns=labels[:], show="headings")
        for i, label_text in enumerate(labels[:]):
            resultados_tree.heading(label_text, text=label_text)
            resultados_tree.column(label_text, width=100)
            resultados_tree.bind("<ButtonRelease-1>", selecionar_resultado)
        resultados_tree.pack(padx=10, pady=10)
        
        # Botão de pesquisa
        buscar_button = ttk.Button(pesquisa_window, text="Buscar", command=buscar_lancamentos)
        buscar_button.pack(padx=10, pady=10)
        
        # Botão de fechar a janela de pesquisa
        fechar_button = ttk.Button(pesquisa_window, text="Fechar", command=pesquisa_window.destroy)
        fechar_button.pack(padx=10, pady=10)
        
    def inserir_lancamentos(root):
        import tkinter as tk
        from tkinter import ttk
        import sqlite3
        from tkinter import filedialog
        import tkinter.messagebox as messagebox
        from PIL import Image, ImageTk
        import pytesseract
        import io
        import os
        import shutil
        import base64  # Importe o módulo base64
        # Se a variável root existir, destrua a janela atual
        if root:
            root.destroy()
        

        # Variáveis globais para as entradas de dados
        local_entry = None
        colaborador_entry = None
        setor_entry = None
        cargo_entry = None
        data_solicitacao_entry = None
        hora_solicitacao_entry = None
        responsavel_entry = None
        nro_nf_entry = None
        data_compra_entry = None
        vl_total_entry = None
        itens_comprados_entry = None
        vl_item_entry = None
               
        def obter_proximo_id_lan():
        # Tente abrir o arquivo para leitura
            try:
                with open("ultimo_id_lan.txt", "r") as file:
                    ultimo_id_lan = int(file.read())
            except FileNotFoundError:
                ultimo_id_lan = 0

            # Incremente o último ID_LAN
            proximo_id_lan = ultimo_id_lan + 1

            # Salve o próximo ID_LAN no arquivo
            with open("ultimo_id_lan.txt", "w") as file:
                file.write(str(proximo_id_lan))

            # Formate o ID_LAN como uma string com zeros à esquerda (comprimento total de 5 dígitos)
            return f"{proximo_id_lan:05}"


        # Função para selecionar o arquivo de imagem
        def selecionar_imagem():
            file_path = filedialog.askopenfilename(filetypes=[("Imagens", "*.jpg *.jpeg *.png *.bmp *.gif")])
            if file_path:
                imagem_entry.delete(0, tk.END)
                imagem_entry.insert(0, file_path)


        def mostrar_mensagem(mensagem):
            messagebox.showinfo("Mensagem", mensagem)

        # Função para inserir dados no banco de dados e associar a imagem
        def inserir_dados():
            # Converte a imagem em base64 e obtém o próximo ID_LAN
            imagem_path = imagem_entry.get()
            if os.path.exists(imagem_path):
                with open(imagem_path, "rb") as f:
                    imagem_base64 = base64.b64encode(f.read()).decode()

                proximo_id_lan = obter_proximo_id_lan()

                # Crie um nome de arquivo para a imagem usando ID_LAN e a data atual
                import datetime
                data_atual = datetime.datetime.now().strftime("%d%m%Y%H%M%S")
                nome_arquivo = f"Comprovantes/{proximo_id_lan}_{data_atual}.jpg"  # Altere a extensão conforme necessário

                # Salve a imagem com o novo nome de arquivo
                with open(nome_arquivo, "wb") as f:
                    f.write(base64.b64decode(imagem_base64))

                # Insira os dados no banco de dados
                dados = (
                    proximo_id_lan,  # Usando o novo ID_LAN
                    local_entry.get(),
                    colaborador_entry.get(),
                    setor_entry.get(),
                    cargo_entry.get(),
                    data_solicitacao_entry.get(),
                    hora_solicitacao_entry.get(),
                    responsavel_entry.get(),
                    nro_nf_entry.get(),
                    data_compra_entry.get(),
                    vl_total_entry.get(),
                    itens_comprados_entry.get(),
                    vl_item_entry.get(),
                    os.path.basename(nome_arquivo),  # Insira o nome do arquivo na coluna "Comprovante"
                )

                conn.execute('''
                    INSERT INTO controle_caixa (
                        ID_LAN, Local, Colaborador, Setor, Cargo, Data_Solicitacao, Hora_Solicitacao,
                        Responsavel, Nro_NF, Data_Compra, VL_Total, Itens_Comprados, VL_Item,
                        Comprovante
                    ) VALUES (?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?)
                ''', dados)

                conn.commit()
                limpar_campos()

                # Limpar o campo de entrada de imagem
                imagem_entry.delete(0, tk.END)

                # Mostrar uma mensagem de validação
                mostrar_mensagem("Dados inseridos com sucesso!")
            else:
                mostrar_mensagem("O arquivo de imagem não foi encontrado.")

            

        # Função para limpar os campos de entrada após a inserção de dados
        def limpar_campos():
            for entry in entries:
                entry.delete(0, tk.END)
            imagem_label.config(image=None)

        # Função para listar todos os dados do banco de dados
        def listar_dados():
            # Limpar qualquer dado existente no Treeview
            for row in tree.get_children():
                tree.delete(row)

            # Executar uma consulta SQL para obter todos os registros
            cursor = conn.execute('SELECT * FROM controle_caixa')
            for row in cursor.fetchall():
                # Inserir os dados no Treeview
                # Certifique-se de que os valores estejam nas colunas corretas
                tree.insert("", "end", values=row[1:])
                
        # Configuração da janela principal
        root = tk.Tk()
        root.title("Controle de Caixa")
        root.state('zoomed')  # Abre a janela maximizada


        # Conectar ao banco de dados
        conn = conectar_ao_banco()

        # Cabeçalho da imagem
        imagem_label = ttk.Label(root)
        imagem_label.place(x=10, y=10, width=400, height=300)

        # Entradas de dados
        entries = []
        labels = ["ID Solicitaçao","Local", "Colaborador", "Setor", "Cargo", "Data Solicitação", "Hora Solicitação",
                "Responsável", "Nro NF", "Data Compra", "VL Total", "Itens Comprados", "VL Item","Comprovante"]

        row1_x = 10
        row2_x = 330
        row1_y = 10
        row2_y = 10  # Ajustado o alinhamento

        # Loop onde você cria as entradas de dados e as associa a variáveis globais
        for i, label_text in enumerate(labels[:6]):
            label = ttk.Label(root, text=label_text)
            label.place(x=row1_x, y=row1_y + i * 30, width=100, height=20)
            entry = ttk.Entry(root)
            entry.place(x=row1_x + 110, y=row1_y + i * 30, width=200, height=20)
            entries.append(entry)
            
            # Atribua as entradas às variáveis globais
            if i == 0:
                local_entry = entry
            elif i == 1:
                colaborador_entry = entry
            elif i == 2:
                setor_entry = entry
            elif i == 3:
                cargo_entry = entry
            elif i == 4:
                data_solicitacao_entry = entry
            elif i == 5:
                hora_solicitacao_entry = entry

        for i, label_text in enumerate(labels[6:]):
            label = ttk.Label(root, text=label_text)
            label.place(x=row2_x, y=row2_y + i * 30, width=100, height=20)
            entry = ttk.Entry(root)
            entry.place(x=row2_x + 110, y=row2_y + i * 30, width=200, height=20)
            entries.append(entry)
            
            # Atribua as entradas às variáveis globais
            if i == 0:
                responsavel_entry = entry
            elif i == 1:
                nro_nf_entry = entry
            elif i == 2:
                data_compra_entry = entry
            elif i == 3:
                vl_total_entry = entry
            elif i == 4:
                itens_comprados_entry = entry
            elif i == 5:
                vl_item_entry = entry

        # Botões à direita dos campos de seleção
        button_x = 1000

        inserir_imagem_button = ttk.Button(root, text="Selecionar Imagem", command=selecionar_imagem)
        inserir_imagem_button.place(x=button_x, y=10, width=200, height=30)  # Ajustado o alinhamento

        inserir_dados_button = ttk.Button(root, text="Inserir Dados", command=inserir_dados)
        inserir_dados_button.place(x=button_x, y=50, width=200, height=30)

        listar_dados_button = ttk.Button(root, text="Listar Dados", command=listar_dados)
        listar_dados_button.place(x=button_x, y=90, width=200, height=30)
        
        voltar_button = ttk.Button(root, text="Voltar", command=lambda: fechar_janela_e_main(root))
        voltar_button.place(x=button_x, y=130, width=200, height=30)    
        
    


        # Rótulo (label) para "Local Imagem"
        local_imagem_label = ttk.Label(root, text="Local Imagem")
        local_imagem_label.place(x=row1_x, y=row1_y + 6 * 30, width=100, height=20)

        # Entrada de dados para o endereço da imagem
        imagem_entry = ttk.Entry(root)
        imagem_entry.place(x=row1_x + 110, y=row1_y + 6 * 30, width=200, height=20)

        # Treeview para listar dados
        tree = ttk.Treeview(root, columns=labels[:], show="headings")
        for i, label_text in enumerate(labels[:]):
            tree.heading(label_text, text=label_text)
            tree.column(label_text, width=100)
        tree.place(x=10, y=220, width=1500, height=200)
        
        root.mainloop()
           
    def dashboards_financeiro():
        root.destroy()  # Fecha a tela de boas-vindas
        # Implemente a lógica para exibir dashboards financeiros aqui
        print("Você selecionou Dashboards Financeiro")
    

    # Configuração da janela principal
    global root
    root = tk.Tk()
    root.title("Bem-vindo ao Controle de Caixa")
    root.protocol("WM_DELETE_WINDOW", fechar_janela)
    

    # Crie um rótulo de boas-vindas
    welcome_label = ttk.Label(root, text="Bem-vindo ao Controle de Caixa", font=("Arial", 16))
    welcome_label.pack(pady=20)

    # Crie botões para as opções
    consultar_button = ttk.Button(root, text="1 - Consultar/Buscar Lançamentos", command=consultar_lancamentos)
    consultar_button.pack(pady=10)

    inserir_button = ttk.Button(root, text="2 - Inserir Lançamentos", command=lambda: inserir_lancamentos(root))
    inserir_button.pack(pady=10)

    dashboard_button = ttk.Button(root, text="3 - Dashboards Financeiro", command=dashboards_financeiro)
    dashboard_button.pack(pady=10)

    root.mainloop()
    
def fechar_janela(root):
    if 'root' in globals():
        root.destroy()    
        
def fechar_janela_e_main(root):
    fechar_janela(root)  # Fecha a janela atual
    main()  # Chama a função main para reiniciar o script
    
if __name__ == "__main__":
    main()

